<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [2]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [3]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.21.5
Pandas:  1.4.2


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

In [36]:
bikes = pd.read_table('bikeshare.csv',header=None)
print(bikes.head())
# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

                                                   0
0  instant,dteday,season,yr,mnth,hr,holiday,weekd...
1  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...
2  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...
3  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...
4  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...
  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
0  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...     
1  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...     
2  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...     
3  5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,...     
4  6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,...     

  instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...                                                                   
1  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...                                         

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [37]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv',header=None)

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [38]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)
    

In [39]:
df = pd.read_excel('Iris.xls', sheet_name = 'Iris')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [40]:
#ANSWER
df = pd.read_excel('Iris.xls', sheet_name= 'Iris', header = 0, usecols = 'A:C', skiprows =[1,4],nrows = 100, skipfooter =0)
df

,Species_No,Petal_width,Petal_length
0,1,0.2,1.4
1,1,0.2,1.3
2,1,0.2,1.4
3,1,0.4,1.7
4,1,0.3,1.4
...,...,...,...
94,2,1.3,4.2
95,2,1.3,4.3
96,2,1.1,3.0
97,2,1.3,4.1


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [22]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-06-02T10:51:30.621Z,58.5233,-153.6854,66.3,2.9,ml,NaN,NaN,NaN,0.62,...,2022-06-02T10:59:56.596Z,"68 km NW of Aleneva, Alaska",earthquake,NaN,0.6,NaN,NaN,automatic,ak,ak
1,2022-06-02T10:51:00.795Z,58.1843,-156.3957,168.4,3.0,ml,NaN,NaN,NaN,0.50,...,2022-06-02T10:53:17.587Z,"57 km E of Egegik, Alaska",earthquake,NaN,0.5,NaN,NaN,automatic,ak,ak


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [3]:
url = 'https://www.ccra.com/airport-codes/'

df = pd.read_html(url)
df

[                City      Country Code
 0            Aalborg      Denmark  AAL
 1           Aalesund       Norway  AES
 2             Aarhus      Denmark  AAR
 3     Abbotsford, BC       Canada  YXX
 4     Abbotsford, BC       Canada  YXX
 ...              ...          ...  ...
 1574        Zanzibar     Tanzania  ZNZ
 1575        Zaragoza        Spain  ZAZ
 1576       Zhengzhou        China  CGO
 1577        Zhoushan        China  HSN
 1578          Zurich  Switzerland  ZRH
 
 [1579 rows x 3 columns]]

In [4]:
df[0]

,City,Country,Code
0,Aalborg,Denmark,AAL
1,Aalesund,Norway,AES
2,Aarhus,Denmark,AAR
3,"Abbotsford, BC",Canada,YXX
4,"Abbotsford, BC",Canada,YXX
...,...,...,...
1574,Zanzibar,Tanzania,ZNZ
1575,Zaragoza,Spain,ZAZ
1576,Zhengzhou,China,CGO
1577,Zhoushan,China,HSN


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [5]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [41]:
#ANSWER
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17380 entries, 0 to 17379
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       17380 non-null  object
 1   1       17380 non-null  object
 2   2       17380 non-null  object
 3   3       17380 non-null  object
 4   4       17380 non-null  object
 5   5       17380 non-null  object
 6   6       17380 non-null  object
 7   7       17380 non-null  object
 8   8       17380 non-null  object
 9   9       17380 non-null  object
 10  10      17380 non-null  object
 11  11      17380 non-null  object
 12  12      17380 non-null  object
 13  13      17380 non-null  object
 14  14      17380 non-null  object
 15  15      17380 non-null  object
 16  16      17380 non-null  object
dtypes: object(17)
memory usage: 2.3+ MB


What are the column names in `bikes`?

In [42]:
#ANSWER
bikes.head

<bound method NDFrame.head of             0           1       2   3     4   5        6        7   \
0      instant      dteday  season  yr  mnth  hr  holiday  weekday   
1            1  2011-01-01       1   0     1   0        0        6   
2            2  2011-01-01       1   0     1   1        0        6   
3            3  2011-01-01       1   0     1   2        0        6   
4            4  2011-01-01       1   0     1   3        0        6   
...        ...         ...     ...  ..   ...  ..      ...      ...   
17375    17375  2012-12-31       1   1    12  19        0        1   
17376    17376  2012-12-31       1   1    12  20        0        1   
17377    17377  2012-12-31       1   1    12  21        0        1   
17378    17378  2012-12-31       1   1    12  22        0        1   
17379    17379  2012-12-31       1   1    12  23        0        1   

               8           9     10      11    12         13      14  \
0      workingday  weathersit  temp   atemp   hum  windsp

What are the data types of these columns?

In [43]:
#ANSWER
bikes.dtypes

0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
dtype: object

What is the (row) index for this DataFrame?

In [25]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17380, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [44]:
#ANSWER
bikes.columns = bikes.iloc[0]
bikes['season']

0        season
1             1
2             1
3             1
4             1
          ...  
17375         1
17376         1
17377         1
17378         1
17379         1
Name: season, Length: 17380, dtype: object

In [45]:
bikes[['season']]

,season
0,season
1,1
2,1
3,1
4,1
...,...
17375,1
17376,1
17377,1
17378,1


How would we use object notation to show the first 4 rows of `atemp`?

In [46]:
#ANSWER
bikes.temp.iloc[0:4]

0    temp
1    0.24
2    0.22
3    0.22
Name: temp, dtype: object

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [39]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: 0

What is the correct way to access the 1st row of the DataFrame by its index?

In [35]:
#ANSWER
bikes.iloc[0]
bikes.iloc[[0]]


,"instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt"
0,"1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,..."


What is the correct way to access the 2nd column of the DataFrame by its index?

In [88]:
#ANSWER
bikes.iloc[:,1]

pandas.core.series.Series

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [89]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [9]:
#ANSWER
bikes.isnull().sum()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt    0
dtype: int64

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [7]:
#ANSWER
bikes.isnull().any()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt    False
dtype: bool

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [65]:
#ANSWER
def scalar_division(data,number):
    data = np.nan_to_num(data,nan=0)
    return data/number

a = np.array([6,np.nan, 3])
b = scalar_division(a,3)
print('Replacing nan:\nBefore',a,'\nAfter', b)

Replacing nan:
Before [ 6. nan  3.] 
After [2. 0. 1.]


Apply the Pandas `isna` function to the following data objects:

In [66]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [68]:
#ANSWER
x = np.isnan(x)
y = np.isnan(y)
print(x,y)

False True


In [69]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [70]:
#ANSWER
z = np.isnan(array)
print(z)

[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

? ANSWER: 

In [76]:
bikes['test'] = 'not_available'
bikes.to_csv('bikes_test.csv')
bikes_test = pd.read_csv('bikes_test.csv', na_values = 'not_available')
bikes_test.head()


,Unnamed: 0,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,test
0,0,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,NaN
1,1,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16,NaN
2,2,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40,NaN
3,3,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32,NaN
4,4,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13,NaN


## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [4]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv',header=0)
bikes.season.nunique()

4

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [5]:
#ANSWER
bikes.head()
print(bikes.loc[:,['instant','dteday','windspeed']])

       instant      dteday  windspeed
0            1  2011-01-01     0.0000
1            2  2011-01-01     0.0000
2            3  2011-01-01     0.0000
3            4  2011-01-01     0.0000
4            5  2011-01-01     0.0000
...        ...         ...        ...
17374    17375  2012-12-31     0.1642
17375    17376  2012-12-31     0.1642
17376    17377  2012-12-31     0.1642
17377    17378  2012-12-31     0.1343
17378    17379  2012-12-31     0.1343

[17379 rows x 3 columns]


Compute and print the overall minimum and maximum of the numeric data columns:

In [6]:
bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'str'

In [7]:
bikes_min, bikes_max = (min(bikes.min().drop('dteday')), max(bikes.max().drop('dteday')))
bikes_min, bikes_max

(0, 17379)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [8]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [9]:
#ANSWER
bikes.atemp.quantile([0.1,0.25,0.5,0.75])

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [10]:
type(bikes['atemp'])

pandas.core.series.Series

In [11]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
13638,13639,2012-07-27,3,1,7,11,0,5,1,1,0.84,0.7727,0.47,0.3284,67,184,251
12096,12097,2012-05-24,2,1,5,5,0,4,1,2,0.60,0.5455,0.88,0.1642,1,29,30
11927,11928,2012-05-17,2,1,5,4,0,4,1,1,0.60,0.6061,0.64,0.3284,3,13,16
5383,5384,2011-08-17,3,0,8,5,0,3,1,1,0.64,0.6061,0.73,0.0000,0,28,28
7214,7215,2011-11-02,4,0,11,5,0,3,1,1,0.30,0.3333,0.81,0.0000,0,27,27


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [12]:
atemp_level = pd.cut(bikes['atemp'], bins = 4,labels=['very cold','cold','mild','hot'])         

What is `atemp_level`?

In [13]:
#ANSWER
atemp_level.unique()


['cold', 'very cold', 'mild', 'hot']
Categories (4, object): ['very cold' < 'cold' < 'mild' < 'hot']

Here is a random sample of `atemp_level`:

In [14]:
atemp_level.sample(5)          

10389    mild
13254    mild
126      cold
10265    mild
16862    cold
Name: atemp, dtype: category
Categories (4, object): ['very cold' < 'cold' < 'mild' < 'hot']

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [15]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

9078     cool
11906    warm
12697    mild
7491     mild
3867     warm
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [19]:
#ANSWER
bikes['atemp_level'] = atemp_level
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
8912,8913,2012-01-12,1,1,1,5,0,4,1,3,0.34,0.3485,0.93,0.0896,0,16,16,mild
14594,14595,2012-09-05,3,1,9,7,0,3,1,2,0.70,0.6667,0.79,0.2537,11,489,500,warm
4508,4509,2011-07-11,3,0,7,18,0,1,1,1,0.86,0.8333,0.53,0.3881,57,415,472,hot
8655,8656,2012-01-01,1,1,1,10,0,0,0,1,0.30,0.3182,0.81,0.1045,18,52,70,mild
5344,5345,2011-08-15,3,0,8,14,0,1,1,1,0.74,0.6818,0.55,0.0896,56,95,151,warm


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [20]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [21]:
# Cleaning Data
print(a,a.shape)
print(b,b.shape)
print(c,c.shape)
print(d,d.shape)
print(e,e.shape)


[0. 0. 0.] (3,)
[[1. 1. 1.]] (1, 3)
[[[7 7 5 8]
  [9 9 5 1]
  [6 2 6 2]]

 [[2 5 5 4]
  [4 8 8 5]
  [1 5 6 9]]] (2, 3, 4)
[0 1 2 3] (4,)
[[1 2 3 4]
 [5 6 7 8]] (2, 4)


## Load Data

Load rock.csv and clean the dataset.

## Check Column Names

Check column names and clean.

## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

## Apply these functions to your dataset

## Describe the new float-only DataFrame.

>




---



---



> > > > > > > > > © 2022 Institute of Data


---



---



